In [ ]:
import pandas as pd

In [ ]:
original_train = pd.read_csv('QuoraQuestions/train.csv')
original_test = pd.read_csv('QuoraQuestions/test.csv')

In [3]:
original_train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [7]:
original_test.head()

,qid,question_text
0,00014894849d00ba98a9,My voice range is A2-C5. My chest voice goes u...
1,000156468431f09b3cae,How much does a tutor earn in Bangalore?
2,000227734433360e1aae,What are the best made pocket knives under $20...
3,0005e06fbe3045bd2a92,Why would they add a hypothetical scenario tha...
4,00068a0f7f41f50fc399,What is the dresscode for Techmahindra freshers?


In [9]:
len(original_train[original_train['target'] == 1])

80810

In [16]:
original_train[original_train['target'] == 1].head()['question_text'][30] # definitively insincere

'Which babies are more sweeter to their parents? Dark skin babies or light skin babies?'

In [21]:
original_train[original_train['target'] == 1].head()['question_text'][110] # on the cusp - could be considered insincere bc "blacks" is not necessarily PC

"If blacks support school choice and mandatory sentencing for criminals why don't they vote Republican?"

In [22]:
original_train[original_train['target'] == 1].head()['question_text'][114] # lascivious and this is borderline incest

'I am gay boy and I love my cousin (boy). He is sexy, but I dont know what to do. He is hot, and I want to see his di**. What should I do?'

In [20]:
original_train[original_train['target'] == 1].head()['question_text'][115] # definitely insincere

'Which races have the smallest penis?'

In [25]:
original_train[original_train['target'] == 1].tail()['question_text'][1306093] # part 2 - incest

'How is it to have intimate relation with your cousin?'

In [27]:
original_train[original_train['target'] == 1].tail()['question_text'][1306099] # racist towards pakistani people

'Do pakis smell of curry and shit?'

In [26]:
original_train[original_train['target'] == 1].tail()['question_text'][1306094] # provocative and trying to make a statement

'Why is it when singers have lyrics about voices in their head, religious people say they hear god and anyone stating they are being targeted by voice to brain technology? Are they called delusional schizo?'

In [4]:
split_train = list(map(lambda x : x.split(), original_train[original_train['target'] == 1]['question_text']))

In [5]:
split_train[:10]

[['Has',
  'the',
  'United',
  'States',
  'become',
  'the',
  'largest',
  'dictatorship',
  'in',
  'the',
  'world?'],
 ['Which',
  'babies',
  'are',
  'more',
  'sweeter',
  'to',
  'their',
  'parents?',
  'Dark',
  'skin',
  'babies',
  'or',
  'light',
  'skin',
  'babies?'],
 ['If',
  'blacks',
  'support',
  'school',
  'choice',
  'and',
  'mandatory',
  'sentencing',
  'for',
  'criminals',
  'why',
  "don't",
  'they',
  'vote',
  'Republican?'],
 ['I',
  'am',
  'gay',
  'boy',
  'and',
  'I',
  'love',
  'my',
  'cousin',
  '(boy).',
  'He',
  'is',
  'sexy,',
  'but',
  'I',
  'dont',
  'know',
  'what',
  'to',
  'do.',
  'He',
  'is',
  'hot,',
  'and',
  'I',
  'want',
  'to',
  'see',
  'his',
  'di**.',
  'What',
  'should',
  'I',
  'do?'],
 ['Which', 'races', 'have', 'the', 'smallest', 'penis?'],
 ['Why', 'do', 'females', 'find', 'penises', 'ugly?'],
 ['How',
  'do',
  'I',
  'marry',
  'an',
  'American',
  'woman',
  'for',
  'a',
  'Green',
  'Card?',
  'How

In [ ]:
split_train__concatenated = sum(split_train, [])
split_train_concatenated[:10]

In [32]:
import itertools
concatenated_split_train = list(itertools.chain.from_iterable(split_train))

In [36]:
from nltk.corpus import stopwords
s = stopwords.words('english')
concatenated_split_train = list(filter(lambda x : x not in s, concatenated_split_train))  # filter out all stop words (e.g. pronouns, articles)

C:\Users\chris\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  This is separate from the ipykernel package so we can avoid doing imports until


In [42]:
pd.Series(concatenated_split_train).value_counts()[:200] # top 200 words 

Why              32669
I                11235
people           10864
Is                8394
How               7510
What              6105
Do                5654
like              5638
Trump             4701
women             4636
Are               4609
think             3758
If                3688
many              3471
get               3152
would             3121
men               3047
Indian            2964
white             2911
Muslims           2798
Quora             2540
Americans         2473
want              2330
black             2330
hate              2145
Indians           2103
girls             2063
India             2037
sex               2027
make              1924
                 ...  
us                 545
In                 543
year               541
big                541
help               540
My                 537
religion           534
African            534
accept             532
eat                530
claim              528
getting            528
trying     

In [47]:
from sklearn.feature_extraction.text import CountVectorizer
# use vectorizer to one hot encode the top 100 most common words in insincere questions among all the questions
# this will be used in training set, so that at prediction time the algorithm looks for questions with these words